In [376]:
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
from datetime import datetime
from itertools import combinations, permutations
from dateutil.relativedelta import relativedelta

In [377]:
df = pd.read_excel('PORTFOLIO.xlsx')

In [378]:
df_gpd = df.groupby('ticker').agg({'data_ini': 'min', 'data_fin': 'max', 'setor':'last'})

In [379]:
tickers = df.ticker.unique()

In [380]:
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()

In [381]:
# Download data
df_list = []
errors = []

utc_from = df.data_ini.min()
utc_to = df.data_fin.max()

for i, row in df_gpd.iterrows():
    try: 
        rates = mt5.copy_rates_range(row.name, mt5.TIMEFRAME_D1,utc_from, utc_to)

        # criamos a partir dos dados obtidos DataFrame
        rates_frame = pd.DataFrame(rates)
        # convertemos o tempo em segundos no formato datetime
        rates_frame['time']=pd.to_datetime(rates_frame['time'], unit='s')
        rates_frame['ticker'] = row.name

        df_list.append(rates_frame[['time', 'close', 'ticker']])
    except Exception:
        errors.append(row.name)

In [382]:
errors

['BIDI11', 'GNDI3', 'HGTX3', 'LCAM3']

In [383]:
df_prices = pd.concat(df_list).pivot(columns='ticker',  index='time', values='close')

In [384]:
df_prices

ticker,ABEV3,ALPA4,AMER3,ASAI3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,...,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VBBR3,VIIA3,VIVT3,WEGE3,YDUQ3
time,,,,,,,,,,,,,,,,,,,,,
2019-07-01,16.40,20.38,30.84,NaN,42.09,11.08,45.48,22.20,25.01,26.20,...,10.39,14.16,17.90,7.74,41.96,21.37,5.12,34.48,10.37,27.18
2019-07-02,16.69,20.85,30.68,NaN,42.41,10.90,44.90,22.18,25.09,26.04,...,10.37,14.65,17.63,7.65,40.20,20.59,5.45,35.41,10.36,27.67
2019-07-03,17.37,20.85,31.13,NaN,44.16,11.15,45.83,22.60,25.43,26.18,...,10.68,14.83,17.67,7.91,40.12,20.94,5.93,35.64,10.43,27.82
2019-07-04,17.49,21.04,33.03,NaN,46.75,11.54,46.58,23.10,25.98,26.43,...,11.05,15.13,17.81,8.25,40.41,21.38,6.24,36.49,10.67,28.70
2019-07-05,17.49,21.20,34.48,NaN,48.45,11.76,46.43,23.28,26.21,27.04,...,11.02,15.89,17.77,8.45,39.39,21.59,6.35,36.18,10.77,29.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-27,13.72,19.80,14.03,14.70,13.31,11.47,33.13,15.06,18.15,24.92,...,12.60,24.26,12.54,9.25,78.05,16.70,2.20,46.46,25.80,13.96
2022-06-28,13.52,19.69,13.42,14.62,13.00,11.15,33.25,14.93,17.89,25.45,...,12.56,23.78,12.71,9.12,79.45,16.96,2.08,46.24,26.02,13.58
2022-06-29,13.61,19.54,13.44,14.38,12.69,10.95,33.08,14.71,17.58,25.48,...,12.50,23.27,12.56,9.01,78.79,16.90,2.09,45.42,25.90,13.40


In [317]:
df_gpd.loc['ASAI3']

data_ini    2021-07-01 00:00:00
data_fin    2022-07-01 00:00:00
setor                    Varejo
Name: ASAI3, dtype: object

In [319]:
df_prices.ASAI3.dropna()

time
2021-03-01    14.08
2021-03-02    14.29
2021-03-03    14.06
2021-03-04    14.43
2021-03-05    14.79
              ...  
2022-06-27    14.70
2022-06-28    14.62
2022-06-29    14.38
2022-06-30    14.27
2022-07-01    14.60
Name: ASAI3, Length: 335, dtype: float64

In [385]:
df_prices.to_excel('database.xlsx')

In [212]:
a = {data:[] for data in df.data_ini.unique()}

for data in df.data_ini.unique():   
    
    df_sem = df.loc[df.data_ini==data]
    
    for setor in df_sem.setor.unique():
        
        tickers = df_sem.loc[df_sem.setor == setor, 'ticker'].to_list()
        a[data].append({setor:list(permutations(tickers, 2))})

In [367]:
df.data_ini.unique()

array(['2019-07-01T00:00:00.000000000', '2020-01-01T00:00:00.000000000',
       '2020-07-01T00:00:00.000000000', '2021-01-01T00:00:00.000000000',
       '2021-07-01T00:00:00.000000000', '2022-01-01T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [373]:
#index number of starting date
df_prices.reset_index().loc[df_prices.reset_index().time >= df.data_ini.unique()[3]].index[0]

375

In [327]:
permut = {data:[] for data in df.data_ini.unique()}
for data in df.data_ini.unique():   
    
    df_sem = df.loc[df.data_ini==data]
    
    for setor in df_sem.setor.unique():
        
        tickers = df_sem.loc[df_sem.setor == setor, 'ticker'].to_list()
        permut[data].extend(list(permutations(tickers, 2)))

In [329]:
permut

{numpy.datetime64('2019-07-01T00:00:00.000000000'): [('AZUL4', 'GOLL4'),
  ('GOLL4', 'AZUL4'),
  ('ABEV3', 'BRFS3'),
  ('ABEV3', 'JBSS3'),
  ('ABEV3', 'MRFG3'),
  ('BRFS3', 'ABEV3'),
  ('BRFS3', 'JBSS3'),
  ('BRFS3', 'MRFG3'),
  ('JBSS3', 'ABEV3'),
  ('JBSS3', 'BRFS3'),
  ('JBSS3', 'MRFG3'),
  ('MRFG3', 'ABEV3'),
  ('MRFG3', 'BRFS3'),
  ('MRFG3', 'JBSS3'),
  ('CCRO3', 'ECOR3'),
  ('CCRO3', 'EMBR3'),
  ('CCRO3', 'RAIL3'),
  ('CCRO3', 'WEGE3'),
  ('ECOR3', 'CCRO3'),
  ('ECOR3', 'EMBR3'),
  ('ECOR3', 'RAIL3'),
  ('ECOR3', 'WEGE3'),
  ('EMBR3', 'CCRO3'),
  ('EMBR3', 'ECOR3'),
  ('EMBR3', 'RAIL3'),
  ('EMBR3', 'WEGE3'),
  ('RAIL3', 'CCRO3'),
  ('RAIL3', 'ECOR3'),
  ('RAIL3', 'EMBR3'),
  ('RAIL3', 'WEGE3'),
  ('WEGE3', 'CCRO3'),
  ('WEGE3', 'ECOR3'),
  ('WEGE3', 'EMBR3'),
  ('WEGE3', 'RAIL3'),
  ('CYRE3', 'MRVE3'),
  ('MRVE3', 'CYRE3'),
  ('COGN3', 'YDUQ3'),
  ('YDUQ3', 'COGN3'),
  ('BRML3', 'IGTI3'),
  ('BRML3', 'MULT3'),
  ('IGTI3', 'BRML3'),
  ('IGTI3', 'MULT3'),
  ('MULT3', 'BRML3'),
  (

In [328]:
for key, value in permut.items():
    print(key, len(value))

2019-07-01T00:00:00.000000000 298
2020-01-01T00:00:00.000000000 412
2020-07-01T00:00:00.000000000 478
2021-01-01T00:00:00.000000000 494
2021-07-01T00:00:00.000000000 538
2022-01-01T00:00:00.000000000 658


In [226]:
180/3000000

6e-05

In [179]:
comb

{'Aéreas': 'GOLL4',
 'Alimentos e Bebidas': 'MRFG3',
 'Bens Industriais': 'WEGE3',
 'Construção Civil': 'MRVE3',
 'Educação': 'YDUQ3',
 'Exploração de imóveis': 'MULT3',
 'Financeiro': 'SULA11',
 'Mineração e Siderurgia': 'VALE3',
 'Papel e Celulose': 'SUZB3',
 'Petroleo, gás e biocombustíveis': 'VBBR3',
 'Saúde': 'RDOR3',
 'Telecomunicações': 'VIVT3',
 'Utilidade Pública': 'TAEE11',
 'Varejo': 'VIIA3',
 'Locação de veículos': 'RENT3',
 'Tecnologia da Informação': 'TOTS3'}

In [177]:
df

,data_ini,data_fin,ticker,setor
0,2019-06-01,2019-12-31,AZUL4,Aéreas
1,2019-06-01,2019-12-31,GOLL4,Aéreas
2,2019-06-01,2019-12-31,ABEV3,Alimentos e Bebidas
3,2019-06-01,2019-12-31,BRFS3,Alimentos e Bebidas
4,2019-06-01,2019-12-31,JBSS3,Alimentos e Bebidas
...,...,...,...,...
454,2022-01-01,2022-06-01,NTCO3,Varejo
455,2022-01-01,2022-06-01,PCAR3,Varejo
456,2022-01-01,2022-06-01,PETZ3,Varejo
457,2022-01-01,2022-06-01,SOMA3,Varejo


In [133]:
separated = {}
for data in df.data_ini.unique():
    tickers = df.loc[df.data_ini == data, 'ticker'].to_list()
    separated[data] = tickers

separated

{numpy.datetime64('2019-06-01T00:00:00.000000000'): ['AZUL4',
  'GOLL4',
  'ABEV3',
  'BRFS3',
  'JBSS3',
  'MRFG3',
  'CCRO3',
  'ECOR3',
  'EMBR3',
  'RAIL3',
  'WEGE3',
  'CYRE3',
  'MRVE3',
  'COGN3',
  'YDUQ3',
  'BRML3',
  'IGTI3',
  'MULT3',
  'B3SA3',
  'BBAS3',
  'BBDC3',
  'BBDC4',
  'BBSE3',
  'CIEL3',
  'IRBR3',
  'ITSA4',
  'ITUB4',
  'SANB11',
  'BRAP4',
  'CSNA3',
  'GGBR4',
  'GOAU4',
  'USIM5',
  'VALE3',
  'KLBN11',
  'SUZB3',
  'CSAN3',
  'PETR3',
  'PETR4',
  'UGPA3',
  'VBBR3',
  'FLRY3',
  'HYPE3',
  'QUAL3',
  'RADL3',
  'TIMS3',
  'VIVT4',
  'CMIG4',
  'EGIE3',
  'ELET3',
  'ELET6',
  'ENBR3',
  'EQTL3',
  'SBSP3',
  'TAEE11',
  'AMER3',
  'CVCB3',
  'LREN3',
  'MGLU3',
  'NTCO3',
  'PCAR4',
  'VIIA3'],
 numpy.datetime64('2020-01-01T00:00:00.000000000'): ['AZUL4',
  'GOLL4',
  'ABEV3',
  'BRFS3',
  'JBSS3',
  'MRFG3',
  'CCRO3',
  'ECOR3',
  'EMBR3',
  'RAIL3',
  'WEGE3',
  'CYRE3',
  'JHSF3',
  'MRVE3',
  'COGN3',
  'YDUQ3',
  'BRML3',
  'IGTI3',
  'MULT3',
  '